In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 50)

# Use this to see how much memory the dataframes use
from sys import getsizeof

from vivarium_research_prl import datatypes
from vivarium_research_prl.noise import corruption, fake_names, noisify
from vivarium_research_prl.find_kids import datasets, noisify_data

!date
!whoami
!uname -a
!pwd

Sat 11 Mar 2023 12:09:35 AM PST
ndbs
Linux int-slurm-sarchive-p0005 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/model_validation


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def sizemb(obj, mib=False):
    mb = 1024**2 if mib else 1e6
    return getsizeof(obj) / mb

# Find data

In [4]:
project_output_dir = \
    '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = (
    'results/vv_tax_observers'
    '/united_states_of_america/2023_03_06_12_42_55'
    '/final_results/2023_03_06_16_21_11'
)
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -halt $output_dir

total 13G
drwxrwsr-x 11 albrja IHME-Simulationscience 5.0K Mar 10 10:39 ..
-rw-r--r--  1 albrja IHME-Simulationscience 1.5G Mar  6 19:37 tax_dependents_observer.csv.bz2
drwxrwsr-x  2 albrja IHME-Simulationscience 4.0K Mar  6 19:26 .
-rw-r--r--  1 albrja IHME-Simulationscience 4.0G Mar  6 19:23 tax_1040_observer.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience 6.7G Mar  6 18:45 tax_w2_observer.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience 191M Mar  6 17:57 social_security_observer.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience  58M Mar  6 17:55 wic_observer.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience  34M Mar  6 17:54 household_survey_observer_cps.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience 6.5M Mar  6 17:54 household_survey_observer_acs.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience 628M Mar  6 17:54 decennial_census_observer.csv.bz2


# See what columns are in the tax observers

In [6]:
%%time
print(output_dir)
w2 = pd.read_csv(f'{output_dir}/tax_w2_observer.csv.bz2', nrows=1000)
w2

/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/vv_tax_observers/united_states_of_america/2023_03_06_12_42_55/final_results/2023_03_06_16_21_11
CPU times: user 36.8 ms, sys: 0 ns, total: 36.8 ms
Wall time: 36.6 ms


,mailing_address_street_name,mailing_address_po_box,simulant_id,last_name,employer_id,ssn,mailing_address_zipcode,mailing_address_unit_number,employer_street_name,date_of_birth,...,employer_name,employer_zipcode,employer_unit_number,employer_state,mailing_address_state,income,employer_city,mailing_address_street_number,employer_street_number,is_w2
0,e lafayette avnue,0,5616_0,Mayer,1550757,519-82-8940,6037.0,NaN,gtway ct,1986-04-19,...,Quality Discount Funerals,7901.0,NaN,NJ,CT,26195.775840,bridgewater township,6410,210,True
1,drover st,0,5616_1,Merritt,389376,391-88-5491,42701.0,NaN,georgetown rd,1943-12-08,...,Matsuyama Gerald W Bautsch MS NCC,33023.0,NaN,FL,KY,793.475572,dalhousie acres,1660,7932,True
2,drover st,0,5616_1,Merritt,1354620,391-88-5491,42701.0,NaN,balance lane,1943-12-08,...,Venus Nails & Joint,85901.0,NaN,AZ,KY,5330.663699,phoenix,1660,1200,True
3,drover st,0,5616_1,Merritt,1393914,391-88-5491,42701.0,NaN,ashton rd,1943-12-08,...,American Laser & Eyeglasses,53029.0,NaN,WI,KY,2174.945045,cty of kaukauna,1660,30,True
4,NaN,6864,5616_2,Register,1535195,503-61-6149,26003.0,NaN,geyer springs rd,1961-04-17,...,Auto Repair Expert Collision Center,30635.0,NaN,GA,WV,22593.068035,toney,NaN,55,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,tiffin st,0,5616_1271,Collins,1439844,845-60-6369,73135.0,NaN,hummingbird ln,1961-09-05,...,Miguel's Mexican Kitchen,44001.0,NaN,OH,OK,35146.392610,harpersfield,4793,6618,True
996,central st,0,5616_1272,Parker,315892,892-05-0612,36303.0,NaN,n 1000 e,1971-06-24,...,HOLT CAT Sonora Quest Provider Services,8638.0,NaN,NJ,AL,74567.834095,franklin,13667,7759,True
997,central st,0,5616_1272,Parker,839813,892-05-0612,36303.0,NaN,garfield dr,1971-06-24,...,Point Nursery,8075.0,NaN,NJ,AL,31545.191573,mnt laurel township,13667,616,True
998,central st,0,5616_1273,Brown,1697414,563-35-0283,36303.0,NaN,pierce av,1964-07-29,...,Family Practice Concepts,28716.0,NaN,NC,AL,12990.744887,mooresville,13667,4903,True


In [10]:
sizemb(w2)

1.090898

In [7]:
w2.dtypes

mailing_address_street_name       object
mailing_address_po_box             int64
simulant_id                       object
last_name                         object
employer_id                        int64
ssn                               object
mailing_address_zipcode          float64
mailing_address_unit_number       object
employer_street_name              object
date_of_birth                     object
age                              float64
middle_initial                    object
first_name                        object
mailing_address_city              object
employer_name                     object
employer_zipcode                 float64
employer_unit_number              object
employer_state                    object
mailing_address_state             object
income                           float64
employer_city                     object
mailing_address_street_number     object
employer_street_number            object
is_w2                               bool
dtype: object

In [8]:
%%time
print(output_dir)
ten40 = pd.read_csv(f'{output_dir}/tax_1040_observer.csv.bz2', nrows=1000)
ten40

/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/vv_tax_observers/united_states_of_america/2023_03_06_12_42_55/final_results/2023_03_06_16_21_11
CPU times: user 37.5 ms, sys: 12.6 ms, total: 50.1 ms
Wall time: 81.1 ms


,mailing_address_street_name,mailing_address_po_box,simulant_id,joint_filer,last_name,ssn,mailing_address_zipcode,mailing_address_unit_number,date_of_birth,age,middle_initial,first_name,mailing_address_city,housing_type,mailing_address_state,mailing_address_street_number
0,no garrett stre,0,6810_0,False,Bricker,220-09-3318,34231.0,NaN,1961-12-21,58.277178,D,Babette,ormond beach,Standard,FL,2114
1,no garrett stre,0,6810_1,True,Bricker,019-45-4762,34231.0,NaN,1959-10-16,60.460501,E,James,ormond beach,Standard,FL,2114
2,no garrett stre,0,6810_2,False,Bricker,777-24-7265,34231.0,NaN,1996-06-12,23.802605,G,Tyree,ormond beach,Standard,FL,2114
3,no garrett stre,0,6810_3,False,Bricker,166-56-8572,34231.0,NaN,1999-01-29,21.173023,C,Ethan,ormond beach,Standard,FL,2114
4,s 160th street,0,6810_4,False,Page,492-03-6898,93550.0,NaN,1966-04-14,53.964567,K,Craig,la puente,Standard,CA,4360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,hawksbury r,0,6810_995,False,Schleyer,593-48-2784,77006.0,NaN,1996-10-07,23.483054,A,Zackary,austin,Standard,TX,3117
996,hawksbury r,0,6810_996,False,Bridgett,572-66-1918,77006.0,NaN,1996-04-03,23.995057,J,Victor,austin,Standard,TX,3117
997,hawksbury r,0,6810_997,False,Dufault,501-13-9995,77006.0,NaN,1996-08-04,23.656622,K,Joseph,austin,Standard,TX,3117
998,mira vista blvd,0,6810_998,False,Gittins,026-76-3958,38671.0,NaN,1951-09-09,68.560794,N,Mary,gulfport,Standard,MS,1488


In [11]:
sizemb(ten40)

0.757977

In [9]:
ten40.dtypes

mailing_address_street_name       object
mailing_address_po_box             int64
simulant_id                       object
joint_filer                         bool
last_name                         object
ssn                               object
mailing_address_zipcode          float64
mailing_address_unit_number       object
date_of_birth                     object
age                              float64
middle_initial                    object
first_name                        object
mailing_address_city              object
housing_type                      object
mailing_address_state             object
mailing_address_street_number     object
dtype: object

In [13]:
%%time
print(output_dir)
dependents = pd.read_csv(f'{output_dir}/tax_dependents_observer.csv.bz2', nrows=1000)
dependents

/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/vv_tax_observers/united_states_of_america/2023_03_06_12_42_55/final_results/2023_03_06_16_21_11
CPU times: user 47.6 ms, sys: 5.28 ms, total: 52.9 ms
Wall time: 71.6 ms


,mailing_address_street_name,guardian_id,mailing_address_po_box,simulant_id,last_name,ssn,mailing_address_zipcode,mailing_address_unit_number,tax_year,sex,date_of_birth,age,middle_initial,first_name,mailing_address_city,housing_type,mailing_address_state,mailing_address_street_number
0,rocky rdge cir,3254_5,0,3254_6,Jones,332-25-3931,6902.0,NaN,2020,Male,2007-08-03,13.353219,S,Charles,branford,Standard,CT,92
1,ne 7th ave,3254_5,0,3254_7,Jones,248-84-0444,21239.0,NaN,2020,Male,2013-09-10,7.245981,S,Charlie,nrt beach,Standard,MD,358
2,ne 7th ave,3254_5,0,3254_8,Jones,567-64-5494,21239.0,NaN,2020,Male,2016-05-25,4.541136,S,Christian,nrt beach,Standard,MD,358
3,83rd ave nw,3254_12,0,3254_13,Joseph,116-21-3182,21216.0,NaN,2020,Female,2003-10-28,17.116178,S,Brynn,poolesville,Standard,MD,1135
4,durant drive,3254_16,0,3254_18,Hoover,684-72-0545,76504.0,NaN,2020,Female,2005-09-20,15.220131,S,Brooklynn,alvarado,Standard,TX,216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,edgepark roa,3254_4417,0,3254_4419,Harrington,841-76-5524,22303.0,NaN,2020,Male,2005-05-23,15.547670,L,Dawson,manassas,Standard,VA,505
996,wedgewood dr,3254_4422,0,3254_4424,Williams,238-05-4862,13325.0,NaN,2020,Male,2002-07-16,18.402372,L,Blake,syracuse,Standard,NY,121
997,wedgewood dr,3254_4422,0,3254_4425,Williams,893-52-9590,13325.0,NaN,2020,Female,2004-09-10,16.245672,M,Hailey,syracuse,Standard,NY,121
998,wedgewood dr,3254_4422,0,3254_4426,Williams,390-20-6703,13325.0,NaN,2020,Male,2008-10-06,12.175847,B,Sean,syracuse,Standard,NY,121


In [14]:
sizemb(dependents)

0.894594

In [15]:
dependents.dtypes

mailing_address_street_name       object
guardian_id                       object
mailing_address_po_box             int64
simulant_id                       object
last_name                         object
ssn                               object
mailing_address_zipcode          float64
mailing_address_unit_number       object
tax_year                           int64
sex                               object
date_of_birth                     object
age                              float64
middle_initial                    object
first_name                        object
mailing_address_city              object
housing_type                      object
mailing_address_state             object
mailing_address_street_number     object
dtype: object

In [17]:
w2.income.describe()

count      1000.000000
mean      33386.107042
std       47868.310951
min         204.088970
25%        9115.538222
50%       20751.468834
75%       42027.431338
max      952909.929951
Name: income, dtype: float64

In [20]:
w2.income.astype('float32').describe()

count      1000.000000
mean      33386.109375
std       47868.308594
min         204.088974
25%        9115.538330
50%       20751.468750
75%       42027.431641
max      952909.937500
Name: income, dtype: float64

# Try loading full W2 dataset and see if we run out of memory...

Woo hoo, I did it! 20 gigabytes, baby, in 32 minutes. And I had 100G of RAM available.

97_515_223 = 98 M rows.

In [21]:
print(output_dir)
%time df_w2 = datatypes.load_csv_data(f'{output_dir}/tax_w2_observer.csv.bz2', nrows=None)
%time print(sizemb(df_w2), 'MB')
df_w2

/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/vv_tax_observers/united_states_of_america/2023_03_06_12_42_55/final_results/2023_03_06_16_21_11
CPU times: user 30min 28s, sys: 50.7 s, total: 31min 19s
Wall time: 31min 26s
20234.027652
CPU times: user 12.8 s, sys: 388 ms, total: 13.1 s
Wall time: 13.1 s


,mailing_address_street_name,mailing_address_po_box,simulant_id,last_name,employer_id,ssn,mailing_address_zipcode,mailing_address_unit_number,employer_street_name,date_of_birth,...,employer_name,employer_zipcode,employer_unit_number,employer_state,mailing_address_state,income,employer_city,mailing_address_street_number,employer_street_number,is_w2
0,e lafayette avnue,0,5616_0,Mayer,1550757,519-82-8940,6037.0,NaN,gtway ct,1986-04-19,...,Quality Discount Funerals,7901.0,NaN,NJ,CT,26195.775391,bridgewater township,6410,210,True
1,drover st,0,5616_1,Merritt,389376,391-88-5491,42701.0,NaN,georgetown rd,1943-12-08,...,Matsuyama Gerald W Bautsch MS NCC,33023.0,NaN,FL,KY,793.475586,dalhousie acres,1660,7932,True
2,drover st,0,5616_1,Merritt,1354620,391-88-5491,42701.0,NaN,balance lane,1943-12-08,...,Venus Nails & Joint,85901.0,NaN,AZ,KY,5330.663574,phoenix,1660,1200,True
3,drover st,0,5616_1,Merritt,1393914,391-88-5491,42701.0,NaN,ashton rd,1943-12-08,...,American Laser & Eyeglasses,53029.0,NaN,WI,KY,2174.945068,cty of kaukauna,1660,30,True
4,NaN,6864,5616_2,Register,1535195,503-61-6149,26003.0,NaN,geyer springs rd,1961-04-17,...,Auto Repair Expert Collision Center,30635.0,NaN,GA,WV,22593.068359,toney,NaN,55,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97515218,taylor drive,0,7359_1173273,Bilbao,1067998,027-68-7923,72758.0,NaN,tradewinds parkway,1977-12-07,...,Ada Vision Care,63146.0,NaN,MO,AR,2487.733643,maryland heights,906,946,True
97515219,grandy avenue,0,7359_1173275,House,1106380,615-85-4413,77005.0,NaN,sam peck rd,1987-09-05,...,Sinfully Sweet Angel's Body,55014.0,NaN,MN,TX,1892.471802,inver grove heights,5535,199,True
97515220,grandy avenue,0,7359_1173276,House,113140,413-45-3939,77005.0,NaN,deering,1993-12-17,...,Nirvana Bleu Blowout Co Castrol Premium Retailer,56007.0,NaN,MN,TX,1257.463501,minneapolis,5535,154,True
97515221,homefield winds ct,0,7359_1173278,Trofimov,924527,164-51-6923,97206.0,NaN,akron road,1974-12-17,...,Once Upon A Cake,62650.0,NaN,IL,OR,2968.415771,midlothian,3437,4401,True


In [26]:
df_w2.dtypes

mailing_address_street_name      category
mailing_address_po_box           category
simulant_id                        object
last_name                        category
employer_id                         int64
ssn                                object
mailing_address_zipcode          category
mailing_address_unit_number      category
employer_street_name             category
date_of_birth                    category
age                               float32
middle_initial                   category
first_name                       category
mailing_address_city             category
employer_name                    category
employer_zipcode                 category
employer_unit_number             category
employer_state                   category
mailing_address_state            category
income                            float32
employer_city                    category
mailing_address_street_number    category
employer_street_number           category
is_w2                             

In [23]:
sizemb(df_w2.ssn)

6479.47862

In [25]:
%time sizemb(df_w2.simulant_id)

CPU times: user 5.96 s, sys: 408 µs, total: 5.96 s
Wall time: 5.95 s


6623.276374

In [27]:
%time sizemb(df_w2.employer_id)

CPU times: user 131 µs, sys: 4 µs, total: 135 µs
Wall time: 139 µs


780.121928

In [28]:
%time sizemb(df_w2.employer_city)

CPU times: user 5.37 ms, sys: 8.04 ms, total: 13.4 ms
Wall time: 13.4 ms


197.854732

In [29]:
%time sizemb(df_w2.is_w2)

CPU times: user 192 µs, sys: 6 µs, total: 198 µs
Wall time: 204 µs


97.515367

In [30]:
%time sizemb(df_w2.mailing_address_zipcode)

CPU times: user 9.87 ms, sys: 3.93 ms, total: 13.8 ms
Wall time: 13.8 ms


198.154505

In [31]:
%time sizemb(df_w2.income)

CPU times: user 193 µs, sys: 6 µs, total: 199 µs
Wall time: 205 µs


390.061036

# Write function to convert SSN from string to int

- $2^{31} = 2,147,483,648$ (2.1 billion = 10 digits)

- An SSN as int $= 941,624,837$ (942 million = 9 digits)

So we can store SSN's as (signed) int32, with -1 representing "No SSN".

In [34]:
ssn_test = df_w2.ssn.iloc[0:5000]
ssn_test

0       519-82-8940
1       391-88-5491
2       391-88-5491
3       391-88-5491
4       503-61-6149
           ...     
4995            NaN
4996            NaN
4997    346-88-0364
4998    248-12-0145
4999    641-62-4837
Name: ssn, Length: 5000, dtype: object

In [35]:
df_w2.ssn.isna().sum() # 4M rows are NaN

4209908

In [63]:
2**32

4294967296

In [64]:
2**31

2147483648

In [65]:
2**31 / 941624837 # Divide max signed 32-bit integer by a large SSN -- about half as large

2.280614915428362

In [60]:
def ssn_to_int(ssn):
    ssn_int = (
        ssn.str.replace('-', '')
        .fillna(-1)
        .astype('int32')
    )
    return ssn_int

%time ssn_to_int(ssn_test)

CPU times: user 5.79 ms, sys: 73 µs, total: 5.86 ms
Wall time: 5.74 ms


0       519828940
1       391885491
2       391885491
3       391885491
4       503616149
          ...    
4995           -1
4996           -1
4997    346880364
4998    248120145
4999    641624837
Name: ssn, Length: 5000, dtype: int32

In [67]:
%time ssn_int = ssn_to_int(df_w2.ssn)
ssn_int

CPU times: user 29.7 s, sys: 2.32 s, total: 32.1 s
Wall time: 32 s


0           519828940
1           391885491
2           391885491
3           391885491
4           503616149
              ...    
97515218     27687923
97515219    615854413
97515220    413453939
97515221    164516923
97515222    798033189
Name: ssn, Length: 97515223, dtype: int32

In [68]:
(ssn_int == -1).sum() # Good, same number of missing SSNs

4209908

In [69]:
sizemb(ssn_int)

390.061036

In [70]:
# Converting from str to int32 improves memory usage by a factor of 16
sizemb(df_w2.ssn) / sizemb(ssn_int)

16.61144800938282

# Test updated `convert_string_ids_to_ints` function with SSN functionality

In [77]:
ten40.dtypes

mailing_address_street_name       object
mailing_address_po_box             int64
simulant_id                       object
joint_filer                         bool
last_name                         object
ssn                               object
mailing_address_zipcode          float64
mailing_address_unit_number       object
date_of_birth                     object
age                              float64
middle_initial                    object
first_name                        object
mailing_address_city              object
housing_type                      object
mailing_address_state             object
mailing_address_street_number     object
dtype: object

In [79]:
%time datatypes.convert_string_ids_to_ints(ten40)
ten40.dtypes

CPU times: user 10.3 ms, sys: 100 µs, total: 10.4 ms
Wall time: 9.98 ms


mailing_address_street_name       object
mailing_address_po_box             int64
simulant_id                        int64
joint_filer                         bool
last_name                         object
ssn                                int32
mailing_address_zipcode          float64
mailing_address_unit_number       object
date_of_birth                     object
age                              float64
middle_initial                    object
first_name                        object
mailing_address_city              object
housing_type                      object
mailing_address_state             object
mailing_address_street_number     object
dtype: object

In [80]:
ten40.simulant_id

0      6810000000000
1      6810000000001
2      6810000000002
3      6810000000003
4      6810000000004
           ...      
995    6810000000995
996    6810000000996
997    6810000000997
998    6810000000998
999    6810000000999
Name: simulant_id, Length: 1000, dtype: int64

In [81]:
ten40.ssn

0      220093318
1       19454762
2      777247265
3      166568572
4      492036898
         ...    
995    593482784
996    572661918
997    501139995
998     26763958
999    402248247
Name: ssn, Length: 1000, dtype: int32

In [82]:
(ten40.ssn == -1).sum()

24

# Convert W2 `simulant_id` and `ssn` columns to ints

In [74]:
df_w2.dtypes

mailing_address_street_name      category
mailing_address_po_box           category
simulant_id                        object
last_name                        category
employer_id                         int64
ssn                                object
mailing_address_zipcode          category
mailing_address_unit_number      category
employer_street_name             category
date_of_birth                    category
age                               float32
middle_initial                   category
first_name                       category
mailing_address_city             category
employer_name                    category
employer_zipcode                 category
employer_unit_number             category
employer_state                   category
mailing_address_state            category
income                            float32
employer_city                    category
mailing_address_street_number    category
employer_street_number           category
is_w2                             

In [83]:
%time print(sizemb(df_w2), 'MB before conversion')
%time datatypes.convert_string_ids_to_ints(df_w2)
%time print(sizemb(df_w2), 'MB after conversion')
df_w2.dtypes

20234.027652 MB before conversion
CPU times: user 13 s, sys: 1.51 s, total: 14.5 s
Wall time: 14.5 s
CPU times: user 3min 19s, sys: 15.9 s, total: 3min 35s
Wall time: 3min 34s
8301.455622 MB after conversion
CPU times: user 775 ms, sys: 716 ms, total: 1.49 s
Wall time: 1.49 s


mailing_address_street_name      category
mailing_address_po_box           category
simulant_id                         int64
last_name                        category
employer_id                         int64
ssn                                 int32
mailing_address_zipcode          category
mailing_address_unit_number      category
employer_street_name             category
date_of_birth                    category
age                               float32
middle_initial                   category
first_name                       category
mailing_address_city             category
employer_name                    category
employer_zipcode                 category
employer_unit_number             category
employer_state                   category
mailing_address_state            category
income                            float32
employer_city                    category
mailing_address_street_number    category
employer_street_number           category
is_w2                             

In [84]:
df_w2

,mailing_address_street_name,mailing_address_po_box,simulant_id,last_name,employer_id,ssn,mailing_address_zipcode,mailing_address_unit_number,employer_street_name,date_of_birth,...,employer_name,employer_zipcode,employer_unit_number,employer_state,mailing_address_state,income,employer_city,mailing_address_street_number,employer_street_number,is_w2
0,e lafayette avnue,0,5616000000000,Mayer,1550757,519828940,6037.0,NaN,gtway ct,1986-04-19,...,Quality Discount Funerals,7901.0,NaN,NJ,CT,26195.775391,bridgewater township,6410,210,True
1,drover st,0,5616000000001,Merritt,389376,391885491,42701.0,NaN,georgetown rd,1943-12-08,...,Matsuyama Gerald W Bautsch MS NCC,33023.0,NaN,FL,KY,793.475586,dalhousie acres,1660,7932,True
2,drover st,0,5616000000001,Merritt,1354620,391885491,42701.0,NaN,balance lane,1943-12-08,...,Venus Nails & Joint,85901.0,NaN,AZ,KY,5330.663574,phoenix,1660,1200,True
3,drover st,0,5616000000001,Merritt,1393914,391885491,42701.0,NaN,ashton rd,1943-12-08,...,American Laser & Eyeglasses,53029.0,NaN,WI,KY,2174.945068,cty of kaukauna,1660,30,True
4,NaN,6864,5616000000002,Register,1535195,503616149,26003.0,NaN,geyer springs rd,1961-04-17,...,Auto Repair Expert Collision Center,30635.0,NaN,GA,WV,22593.068359,toney,NaN,55,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97515218,taylor drive,0,7359001173273,Bilbao,1067998,27687923,72758.0,NaN,tradewinds parkway,1977-12-07,...,Ada Vision Care,63146.0,NaN,MO,AR,2487.733643,maryland heights,906,946,True
97515219,grandy avenue,0,7359001173275,House,1106380,615854413,77005.0,NaN,sam peck rd,1987-09-05,...,Sinfully Sweet Angel's Body,55014.0,NaN,MN,TX,1892.471802,inver grove heights,5535,199,True
97515220,grandy avenue,0,7359001173276,House,113140,413453939,77005.0,NaN,deering,1993-12-17,...,Nirvana Bleu Blowout Co Castrol Premium Retailer,56007.0,NaN,MN,TX,1257.463501,minneapolis,5535,154,True
97515221,homefield winds ct,0,7359001173278,Trofimov,924527,164516923,97206.0,NaN,akron road,1974-12-17,...,Once Upon A Cake,62650.0,NaN,IL,OR,2968.415771,midlothian,3437,4401,True


# Save converted W2 dataframe as an HDF to speed up loading later

In [89]:
!ls /share/scratch/users/ndbs/prl/

2023_02_27_10_25_20  address_id_to_zipcode_2023_02_02_10_16_21.csv.bz2
2023_03_06_12_42_55


In [92]:
hdf_dir = '/share/scratch/users/ndbs/prl/2023_03_06_12_42_55'
!ls -halt $hdf_dir

total 8.0K
drwxrwxr-x 2 ndbs Domain Users    0 Mar 11 16:29 .
drwxrwxr-x 4 ndbs Domain Users 1.5K Mar 11 16:29 ..


In [93]:
%%time
w2_hdf_filepath = f'{hdf_dir}/w2.hdf'
df_w2.to_hdf(
    w2_hdf_filepath, 'w2',
    format='table',
    complevel=5, complib='bzip2'
)

!ls -halt $hdf_dir

total 3.9G
-rw-rw-r-- 1 ndbs Domain Users 3.9G Mar 11 16:46 w2.hdf
drwxrwxr-x 2 ndbs Domain Users  512 Mar 11 16:46 .
drwxrwxr-x 4 ndbs Domain Users 1.5K Mar 11 16:29 ..
CPU times: user 12min 34s, sys: 7.51 s, total: 12min 41s
Wall time: 12min 59s


In [94]:
%%time
temp = pd.read_hdf(w2_hdf_filepath)
temp

CPU times: user 5min 1s, sys: 6.87 s, total: 5min 8s
Wall time: 5min 17s


,mailing_address_street_name,mailing_address_po_box,simulant_id,last_name,employer_id,ssn,mailing_address_zipcode,mailing_address_unit_number,employer_street_name,date_of_birth,...,employer_name,employer_zipcode,employer_unit_number,employer_state,mailing_address_state,income,employer_city,mailing_address_street_number,employer_street_number,is_w2
0,e lafayette avnue,0,5616000000000,Mayer,1550757,519828940,6037.0,NaN,gtway ct,1986-04-19,...,Quality Discount Funerals,7901.0,NaN,NJ,CT,26195.775391,bridgewater township,6410,210,True
1,drover st,0,5616000000001,Merritt,389376,391885491,42701.0,NaN,georgetown rd,1943-12-08,...,Matsuyama Gerald W Bautsch MS NCC,33023.0,NaN,FL,KY,793.475586,dalhousie acres,1660,7932,True
2,drover st,0,5616000000001,Merritt,1354620,391885491,42701.0,NaN,balance lane,1943-12-08,...,Venus Nails & Joint,85901.0,NaN,AZ,KY,5330.663574,phoenix,1660,1200,True
3,drover st,0,5616000000001,Merritt,1393914,391885491,42701.0,NaN,ashton rd,1943-12-08,...,American Laser & Eyeglasses,53029.0,NaN,WI,KY,2174.945068,cty of kaukauna,1660,30,True
4,NaN,6864,5616000000002,Register,1535195,503616149,26003.0,NaN,geyer springs rd,1961-04-17,...,Auto Repair Expert Collision Center,30635.0,NaN,GA,WV,22593.068359,toney,NaN,55,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97515218,taylor drive,0,7359001173273,Bilbao,1067998,27687923,72758.0,NaN,tradewinds parkway,1977-12-07,...,Ada Vision Care,63146.0,NaN,MO,AR,2487.733643,maryland heights,906,946,True
97515219,grandy avenue,0,7359001173275,House,1106380,615854413,77005.0,NaN,sam peck rd,1987-09-05,...,Sinfully Sweet Angel's Body,55014.0,NaN,MN,TX,1892.471802,inver grove heights,5535,199,True
97515220,grandy avenue,0,7359001173276,House,113140,413453939,77005.0,NaN,deering,1993-12-17,...,Nirvana Bleu Blowout Co Castrol Premium Retailer,56007.0,NaN,MN,TX,1257.463501,minneapolis,5535,154,True
97515221,homefield winds ct,0,7359001173278,Trofimov,924527,164516923,97206.0,NaN,akron road,1974-12-17,...,Once Upon A Cake,62650.0,NaN,IL,OR,2968.415771,midlothian,3437,4401,True


In [95]:
%time sizemb(temp)
%time temp.equals(df_w2)

CPU times: user 486 ms, sys: 3.93 ms, total: 490 ms
Wall time: 488 ms
CPU times: user 3.78 s, sys: 1.58 s, total: 5.36 s
Wall time: 5.36 s


True

In [96]:
 sizemb(temp)

9081.501258

In [97]:
del temp

# Look into observation times

In [99]:
df_w2.loc[:5].T

,0,1,2,3,4,5
mailing_address_street_name,e lafayette avnue,drover st,drover st,drover st,NaN,south justine street
mailing_address_po_box,0,0,0,0,6864,0
simulant_id,5616000000000,5616000000001,5616000000001,5616000000001,5616000000002,5616000000003
last_name,Mayer,Merritt,Merritt,Merritt,Register,Williams
employer_id,1550757,389376,1354620,1393914,1535195,17273
ssn,519828940,391885491,391885491,391885491,503616149,540777831
mailing_address_zipcode,6037.0,42701.0,42701.0,42701.0,26003.0,35603.0
mailing_address_unit_number,NaN,NaN,NaN,NaN,NaN,NaN
employer_street_name,gtway ct,georgetown rd,balance lane,ashton rd,geyer springs rd,heritage way
date_of_birth,1986-04-19,1943-12-08,1943-12-08,1943-12-08,1961-04-17,1986-04-30


In [100]:
df_w2.loc[df_w2.simulant_id==5616000000001].T

,1,2,3,832562,1777563,2739187,2739188,2739189,3714364,4697045,5717342,6716465,7721013,8731591
mailing_address_street_name,drover st,drover st,drover st,drover st,drover st,ledgelawn dr,ledgelawn dr,ledgelawn dr,ledgelawn dr,ledgelawn dr,ledgelawn dr,ledgelawn dr,ledgelawn dr,ledgelawn dr
mailing_address_po_box,0,0,0,0,0,0,0,0,0,0,0,0,0,0
simulant_id,5616000000001,5616000000001,5616000000001,5616000000001,5616000000001,5616000000001,5616000000001,5616000000001,5616000000001,5616000000001,5616000000001,5616000000001,5616000000001,5616000000001
last_name,Merritt,Merritt,Merritt,Merritt,Merritt,Merritt,Merritt,Merritt,Merritt,Merritt,Merritt,Merritt,Merritt,Merritt
employer_id,389376,1354620,1393914,389376,389376,1,389376,929942,1,1,1,1,1,1
ssn,391885491,391885491,391885491,391885491,391885491,391885491,391885491,391885491,391885491,391885491,391885491,391885491,391885491,391885491
mailing_address_zipcode,42701.0,42701.0,42701.0,42701.0,42701.0,92505.0,92505.0,92505.0,92505.0,92505.0,92505.0,92505.0,92505.0,92505.0
mailing_address_unit_number,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
employer_street_name,georgetown rd,balance lane,ashton rd,georgetown rd,georgetown rd,n fender ave,georgetown rd,hilltop road,n fender ave,n fender ave,n fender ave,n fender ave,n fender ave,n fender ave
date_of_birth,1943-12-08,1943-12-08,1943-12-08,1943-12-08,1943-12-08,1943-12-08,1943-12-08,1943-12-08,1943-12-08,1943-12-08,1943-12-08,1943-12-08,1943-12-08,1943-12-08


In [101]:
%time sizemb(df_w2.simulant_id)

CPU times: user 416 µs, sys: 18 µs, total: 434 µs
Wall time: 443 µs


780.121928

In [102]:
%time sizemb(df_w2.ssn)

CPU times: user 377 µs, sys: 16 µs, total: 393 µs
Wall time: 402 µs


390.061036

In [103]:
df_w2.dtypes

mailing_address_street_name      category
mailing_address_po_box           category
simulant_id                         int64
last_name                        category
employer_id                         int64
ssn                                 int32
mailing_address_zipcode          category
mailing_address_unit_number      category
employer_street_name             category
date_of_birth                    category
age                               float32
middle_initial                   category
first_name                       category
mailing_address_city             category
employer_name                    category
employer_zipcode                 category
employer_unit_number             category
employer_state                   category
mailing_address_state            category
income                            float32
employer_city                    category
mailing_address_street_number    category
employer_street_number           category
is_w2                             

In [105]:
sizemb(df_w2.middle_initial)

97.517947

In [106]:
%time sizemb(df_w2.middle_initial.astype(str))

CPU times: user 7.25 s, sys: 1.03 s, total: 8.27 s
Wall time: 8.27 s


5655.883078

In [107]:
%time df_w2.middle_initial.astype(str)

CPU times: user 1.45 s, sys: 1.31 s, total: 2.76 s
Wall time: 2.76 s


0           W
1           J
2           J
3           J
4           D
           ..
97515218    S
97515219    J
97515220    C
97515221    W
97515222    A
Name: middle_initial, Length: 97515223, dtype: object

In [108]:
getsizeof('M')

50

In [109]:
getsizeof('W')

50

In [110]:
getsizeof('ab')

51

In [111]:
getsizeof('abcde')

54

In [112]:
getsizeof('')

49

In [115]:
len(df_w2) * 50 /1e6

4875.76115

In [116]:
6*24

144

In [122]:
# 17 object (str) columns at 6 GB each
# + 6 numeric columns at .78 GB each
# + 1 bool column at .0975 GB each
17*6 + 6*.78 + 1*.0975

106.7775

In [118]:
sizemb(df_w2.is_w2)

97.515367

In [120]:
sizemb(df_w2.simulant_id)

780.121928

In [123]:
17*6

102